# **Implémentation du modèle**
---

## Introduction

Ce notebook contient les codes nécessaire à la mise en place du modèle. Le modèle est inspiré de celui de Iacoviello et Navarro : 

- La première étape consiste à identifier les chocs de politique monétaire, en prenant le réside de la régression du taux d'intérêt sur l'écart d'inflation par rapport à la cible et l'output gap ou le chômage. En réalité Iacoviello et Navarro utilisent des lags et valeurs présentes de l'inflation, des spreads de crédit, du PIB et des lags des taux des fonds fédéraux.
Il nous manque donc l'output gap, l'inflation et le chômage de la zone euro
- La deuxième étape consiste à estimer l'impact de ces chocs sur l'activité économique (PIB, emploi ...) en régressant ces variables sur les chocs et des variables de contrôles (4 lags du PIB et des trends linéaires et quadratiques)



## Identification des chocs

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

Problème: dans Iacoviello et Navarro, ils régressent le taux d'intérêt sur le PIB etc des US seulement. Or on est dans la zone euro ici, doit-on faire une régression de panel ? 

Pour faire la première étape il nous faudrait un data frame avec les données de seulement la zone euro.
Il faut construire les variables suivantes : 
- log_euroarea_gdp, 
- corporate spreads, 
- lagged_federal_funds (<span style="color:red"> je pense qu'il faut faire une variable par lag, ils prennent 4 lags dans le papier</span>),
- log_foreign_gdp (<span style="color:red"> le problème c'est qu'on n'a pas de pays étrangers dans notre base donc on ne peut pas vraiment créer cette variable</span> )


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = pd.read_pickle('data_final.pkl')


# Sélection des colonnes utiles
colonnes_utiles = ['federal_funds_rate', 'inflation', 'log_euroarea_gdp', 'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time']
data = data[colonnes_utiles]

# Création d'une variable de tendance quadratique
data['time_squared'] = data['time'] ** 2

# Définition de rt=federal funds rate
Y = data['federal_funds_rate']

# Définir les variables indépendantes
X = data[['inflation', 'log_euroarea_gdp', 'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time', 'time_squared']]
X = sm.add_constant(X)  # Ajout d'une constante pour l'intercept

# Regression
model = sm.OLS(Y, X).fit()

# Extraire les résidus comme chocs monétaires
data['monetary_shocks'] = model.resid

# Afficher le résumé de la régression
print(model.summary())

# Sauvegarder les résultats dans un fichier CSV
data[['time', 'monetary_shocks']].to_csv('identified_monetary_shocks.csv', index=False)


KeyError: "None of [Index(['federal_funds_rate', 'inflation', 'log_euroarea_gdp',\n       'corporate_spreads', 'log_foreign_gdp', 'lagged_federal_funds', 'time'],\n      dtype='object')] are in the [columns]"

In [4]:
data

,CPI_Austria,PIB_Austria,LT_IR_Austria,ST_IR_Austria,WH_Austria,CPI_Belgium,PIB_Belgium,LT_IR_Belgium,ST_IR_Belgium,WH_Belgium,...,CPI_Switzerland,PIB_Switzerland,LT_IR_Switzerland,ST_IR_Switzerland,WH_Switzerland,CPI_United_Kingdom,PIB_United_Kingdom,LT_IR_United_Kingdom,ST_IR_United_Kingdom,WH_United_Kingdom
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,
1995-Q1,NaN,43186.6,7.627367,5.043334,40.00,NaN,53331.5,8.293333,5.706666,37.8,...,NaN,64546.7,5.275333,NaN,NaN,NaN,257589.8,8.661467,6.722410,36.90
1995-Q2,NaN,46786.8,7.184033,4.680000,39.95,NaN,56119.3,7.590000,5.050000,37.8,...,NaN,67655.1,4.947333,NaN,NaN,NaN,248722.5,8.242567,6.742444,36.85
1995-Q3,NaN,46335.5,6.997967,4.373333,39.90,NaN,53741.7,7.186666,4.416667,37.8,...,NaN,67566.8,4.714667,NaN,NaN,NaN,255889.4,8.108933,6.849310,36.80
1995-Q4,NaN,47362.4,6.730933,4.176667,39.85,NaN,57115.3,6.853333,3.960000,37.8,...,NaN,70342.0,4.175667,NaN,NaN,NaN,263308.8,7.788167,6.696187,36.75
1996-Q1,71.553333,44641.2,6.396633,3.476667,39.80,70.003333,54240.0,6.640000,3.350000,37.8,...,NaN,67670.1,4.400333,NaN,37.0,68.1,262343.7,7.753200,6.277534,36.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-Q4,132.593333,122280.3,3.125533,3.957456,32.90,127.550000,158085.7,3.150000,3.957456,34.1,...,106.443333,213788.9,0.841333,1.718106,36.3,NaN,NaN,4.234600,5.326667,NaN
2024-Q1,133.326667,117304.1,2.835333,3.923615,34.00,129.700000,147812.8,2.890000,3.923615,35.0,...,106.703333,212946.8,0.765667,1.628413,35.7,NaN,NaN,4.029300,5.203333,NaN
2024-Q2,134.366667,120746.2,3.013533,3.808172,32.90,131.086667,154840.7,3.063333,3.808172,33.8,...,107.440000,211841.3,0.722333,1.475667,35.9,NaN,NaN,4.201500,5.166667,NaN


## Estimation de l'impact des chocs

Nous allons maintenant estimer avec une régression de panel l'impact des chocs de politique monétaire identifiés, sur différentes variables (GDP, employment ? ...).  
Nous allons estimer une équation de la forme suivante, comme dans Iacoviello & Navarro (2018) :

$$
y_{i,t+h} = \alpha_{i,h} + \beta_h u_t + A_{h,i} Z_{i,t} + \varepsilon_{i,t+h}
$$

où :

- $y_{i,t+h}$ est le PIB du pays $ i $ au temps $t$,
- $\alpha_{i,h} $ est un effet fixe spécifique au pays,
- $u_t$ est le choc monétaire,
- $Z_{i,t}$  représente les variables de contrôle : 4 lags du PIB, des tendances linéaire et quadratique,
- $\varepsilon_{i,t+h}$  est le terme d'erreur.

Pour chaque h il faut estimer un $\beta_h$, ensuite il faut faire les IRF (fonctions de réponse impulsionnelle) pour les différents Y qu'on utilise.


Pour les variables de contrôle, il faut sûrement tester lesquelles sont les meilleures, ne pas forcément reproduire exactement le modèle.




<span style="color:red;"> Le code suivant est fait avec chat gpt entièrement, il faut modifier le nom des variables, je ne sais pa du tout s'il va fonctionner ni si c'est exactement le modèle qu'il faut faire. </span>

Définir la fonction d'estimation

In [ ]:
def local_projection_irf(data, response_var, shock_var, control_vars, max_horizon=12):
    """Estime les réponses impulsionnelles (IRF) par projections locales."""
    
    results = []
    
    for h in range(max_horizon + 1):
        data[f'{response_var}_lead_{h}'] = data.groupby('Country')[response_var].shift(-h)
        
        df = data.dropna(subset=[f'{response_var}_lead_{h}', shock_var] + control_vars)
        
        y = df[f'{response_var}_lead_{h}']
        X = df[[shock_var] + control_vars]
        X = sm.add_constant(X)  # Ajouter l'intercept

        model = sm.OLS(y, X).fit(cov_type="cluster", cov_kwds={"groups": df["Country"]})  # Cluster par pays
        results.append((h, model.params[shock_var], model.bse[shock_var]))  # Stocker horizon, coefficient, erreur standard

    return pd.DataFrame(results, columns=["Horizon", "Beta", "SE"])


Exécuter l'estimation

In [ ]:
control_vars = ['GDP_lag1', 'GDP_lag2', 'GDP_lag3', 'GDP_lag4', 'time', 'time_squared']  # Exemples de contrôles
irf_results = local_projection_irf(data, response_var="GDP", shock_var="Shock", control_vars=control_vars, max_horizon=12)


Tracer les IRF

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(irf_results["Horizon"], irf_results["Beta"], marker="o", label="IRF")
plt.fill_between(irf_results["Horizon"], 
                 irf_results["Beta"] - 1.96 * irf_results["SE"], 
                 irf_results["Beta"] + 1.96 * irf_results["SE"], 
                 color='gray', alpha=0.3, label="IC 95%")
plt.axhline(0, color='black', linestyle='--')
plt.xlabel("Horizon (trimestres)")
plt.ylabel("Effet du choc monétaire")
plt.title("Réponse impulsionnelle du PIB à un choc monétaire")
plt.legend()
plt.show()
